In [1]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
import pandas as pd

### Initializations

In [15]:
#epsilon = 0.01
# h = uniformly refined mesh
# order = k

In [2]:
## Parameter setup
orders = [1, 2, 3]
beta = (2,0.00001)
mesh_size = [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313]
eps = 0.01

# Exact Solution
p = lambda x: x + (exp(beta[0]*x/eps)-1)/(1-exp(beta[0]/eps))
q = lambda y: y + (exp(beta[1]*y/eps)-1)/(1-exp(beta[1]/eps))
exact = p(x) * q(y)

coeff = p(x) + q(y)

coeff_x = p(x)
coeff_y = q(y)

alpha = 100

In [3]:
## Dataframes
columns = ['Order', 'Mesh Size', 'Error']

### Exact Solution for Convection-Diffusion Equation

In [ ]:
orders = 1
mesh_size = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=mesh_size))
fes = L2(mesh, order=orders, dirichlet="bottom|right|left|top")
gfu = GridFunction(fes)
Draw(exact, mesh, 'exact')

### Discontinuous Galerkin for Convection-Diffusion with upwind

In [ ]:
dg = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        fes = L2(mesh, order=order, dgjumps=True)
        u, v = fes.TnT()


        jump_u = u-u.Other()
        jump_v = v-v.Other()
        n = specialcf.normal(2)
        mean_dudn = eps * 0.5*n * (grad(u) + grad(u.Other()))
        mean_dvdn = eps * 0.5*n * (grad(v) + grad(v.Other()))



        h = specialcf.mesh_size
        a = BilinearForm(fes)

        diffusion = eps * grad(u) * grad(v) * dx \
            +alpha * eps * order ** 2/ h * jump_u * jump_v * dx(skeleton=True) \
            +(-mean_dudn * jump_v-mean_dvdn * jump_u) * dx(skeleton=True) \
            +alpha * eps * order ** 2/h * u * v * ds(skeleton=True) \
            + eps * (-n * grad(u) * v -n * grad(v) * u) * ds(skeleton=True)

        a += diffusion
        a.Assemble()

 
        f = LinearForm(fes)
        f += SymbolicLFI(exact * v )
        f.Assemble()


        gfu = GridFunction(fes, name="uDG")
        gfu.vec.data = a.mat.Inverse() * f.vec
        #Draw (gfu)
        
        b = CoefficientFunction((beta[0],beta[1]) )

        uup = IfPos(b*n, u, u.Other())

        convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dx(skeleton=True)+ IfPos(b*n,b*n,0)*u*v*ds(skeleton=True)

        acd = BilinearForm(fes)
        acd += diffusion + convection

        acd.Assemble()

        gfu = GridFunction(fes)
        gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
        
        error = sqrt (Integrate ((gfu-exact)*(gfu-exact), mesh))
        dg.loc[len(dg)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)
    print('......................................................................')
    

%store dg

### Hybrid Discontinuous Galerkin Convection-Diffusion with upwind

In [ ]:
hdg = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order)
        F = FacetFESpace(mesh, order=order, dirichlet="bottom|left|right|top")

        fes = FESpace([V,F])
        u,uhat = fes.TrialFunction()
        v,vhat = fes.TestFunction()

        jump_u = u-uhat
        jump_v = v-vhat

        condense = True
        h = specialcf.mesh_size
        n = specialcf.normal(mesh.dim)

        a = BilinearForm(fes, condense=condense)
        dS = dx(element_boundary=True)

        diffusion = eps * grad(u) * grad(v) * dx + \
            alpha * eps * order ** 2/h * jump_u * jump_v * dS + \
            eps * (-grad(u) * n * jump_v - grad(v) * n * jump_u) * dS

                     
        b = CoefficientFunction((beta[0],beta[1])) 

        uup = IfPos(b * n, u, uhat)
              
        convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dS

        a += convection + diffusion
        
        a.Assemble()
                
        f = LinearForm(fes)
        f += SymbolicLFI(exact * v)
        f.Assemble()

        gfu = GridFunction(fes)
        
        if not condense:
            inv = a.mat.Inverse(fes.FreeDofs(), "umfpack")
            gfu.vec.data = inv * f.vec
        else:
            f.vec.data += a.harmonic_extension_trans * f.vec

            inv = a.mat.Inverse(fes.FreeDofs(True), "umfpack")
            gfu.vec.data = inv * f.vec

            gfu.vec.data += a.harmonic_extension * gfu.vec
            gfu.vec.data += a.inner_solve * f.vec
            
        error = sqrt (Integrate ((gfu.components[0] - exact)*(gfu.components[0]- exact), mesh))
        hdg.loc[len(hdg)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)                            
    print('......................................................................')

%store hdg

### Solving Mass Matrix problem

In [ ]:
mm = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        fes = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        u, v = fes.TnT()
             
        m = BilinearForm(fes, symmetric=False)
        m += u * v * dx
        m.Assemble()
        
        f = LinearForm(fes)
        f += SymbolicLFI(exact * v)
        f.Assemble()
        
        invmstar = m.mat.Inverse()
        
        gfu = GridFunction(fes)        
        gfu.vec.data = invmstar * f.vec
        #Draw(gfu, mesh, 'test')
        
        error = sqrt (Integrate ((gfu-exact)*(gfu-exact), mesh))
        mm.loc[len(mm)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)            
    print('......................................................................')

%store mm

### Enrichment

#### Wall function Enrichment with Mass Matrix

###### Wall function Enrichment with Mass Matrix (enrich x-axis)

In [ ]:
emm_x = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        
        fes = FESpace([V, Qx])

        (us, px), (vs, qx) = fes.TnT()
        
        p = coeff_x * px
        q = coeff_x * qx

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_x.loc[len(emm_x)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    

%store emm_x

###### Wall function Enrichment with Mass Matrix (enrich y-axis)

In [ ]:
emm_y = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_y = BitArray(Q.ndof)        
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qy = Compress(Q, active_dofs=ba_y)
        fes = FESpace([V, Qy])

        (us, py), (vs, qy) = fes.TnT()
        
        p = coeff_y * py
        q = coeff_y * qy

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_y.loc[len(emm_y)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    
%store emm_y

###### Wall function Enrichment with Mass Matrix (enrich both xy-axis)

In [ ]:
emm_xy = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)        
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy])

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        p = coeff_x * px + coeff_y * py
        q = coeff_x * qx + coeff_y * qy

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_xy.loc[len(emm_xy)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    

%store emm_xy

###### Effect of Integraton Rule Values on Convergence

In [ ]:
for val0 in range(1, 10, 1):
    for val1 in range(1, 10, 1):
        for order in orders:
            for size in mesh_size:
                eps = size
                #eps = 1e-8
                mesh = Mesh(unit_square.GenerateMesh(maxh=size))
                V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
                Q = L2(mesh, order=0)

                ba_x = BitArray(Q.ndof)        
                ba_x.Clear()
                for el in Q.Elements():
                    mark = False
                    for v in el.vertices:
                        if (mesh[v].point[0] > 1-eps):
                            mark = True
                    for dof in el.dofs:
                        ba_x[dof] = mark

                ba_y = BitArray(Q.ndof)        
                ba_y.Clear()
                for el in Q.Elements():
                    mark = False
                    for v in el.vertices:
                        if (mesh[v].point[1] > 1-eps):
                            mark = True
                    for dof in el.dofs:
                        ba_y[dof] = mark

                Qx = Compress(Q, active_dofs=ba_x)
                Qy = Compress(Q, active_dofs=ba_y)

                fes = FESpace([V, Qx, Qy])

                (us, px, py), (vs, qx, qy) = fes.TnT()

                p = coeff_x * px + coeff_y * py
                q = coeff_x * qx + coeff_y * qy

                u = us + p
                v = vs + q

                m = BilinearForm(fes, symmetric=False)
                m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,val0*order))
                m.Assemble()

                f = LinearForm(fes)
                f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,val1*order))
                f.Assemble()

                invmstar = m.mat.Inverse(inverse="sparsecholesky")

                gfu = GridFunction(fes)
                gfu.vec.data = invmstar * f.vec

                u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
                print('val0: ', val0, 'val1: ', val1, 'Order:', order, ' Mesh Size:', size , "L2 Error:", sqrt (Integrate ((u2 - exact)*(u2- exact), mesh)))
        #print('val0: ', val0, 'val1: ', val1, 'Order:', order, ' Mesh Size:', size , "L2 Error:", sqrt (Integrate ((u2 - exact)*(u2- exact), mesh)))
    print('.........................................................................')

###### Testing with different preconditioners (TODO)

In [ ]:
#print(m.mat)
#print(fes.FreeDofs())
#print(V.ndof)
#print(Q.ndof)
#print(gfu.vec)

In [ ]:
orders = [1, 2, 3, 4]
#orders = [4]
#mesh_size = [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313]
mesh_size = [0.25]
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")

        Q = L2(mesh, order=0)
        ba = BitArray(mesh.ne)
        ba.Set()
        ba[0] = False
        # ba[0] = False
        
        Q = Compress(Q, active_dofs=ba)
        fes = FESpace([V, Q])

        #gfu = GridFunction(fes)

        (u, p), (v, q) = fes.TnT()

        p = coeff * p
        q = coeff * q

        grad_u = grad(u) + CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) * p
        grad_v = grad(v) + CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) * q
        
        u = u + p
        v = v + q

        b = CoefficientFunction((beta[0],beta[1]))        

        a = BilinearForm(fes, symmetric=False)
        a += SymbolicBFI((eps *  grad_u * grad_v) + (b * grad_u * v)).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        a.Assemble()

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        mstar = m.mat.CreateMatrix()
        mstar.AsVector().data = m.mat.AsVector() + a.mat.AsVector()
        invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

        gfu = GridFunction(fes)
        res = gfu.vec.CreateVector()

        res.data = f.vec  - a.mat * gfu.vec
        gfu.vec.data = invmstar * res
        u2 = gfu.components[0] + gfu.components[1] * coeff
        Draw(u2, mesh, 'test')

        print('Order:', order, ' Mesh Size:', size , "L2 Error:", sqrt (Integrate ((u2 - exact)*(u2- exact), mesh)))
    print('...........................................................')

#### Wall function Enrichment with DG

In [5]:
edg = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dgjumps=True)
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        ## Does the dgjumps affect anything here?
        fes = FESpace([V, Qx, Qy], dgjumps = True)

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        p = (coeff_x * px) + (coeff_y * py)
        q = (coeff_x * qx) + (coeff_y * qy)

        u = us + p
        v = vs + q
        
        u_Other = us.Other() + px.Other()
        v_Other = vs.Other() + qx.Other()
        
        jump_u = u - u_Other
        jump_v = v - v_Other
        
        n = specialcf.normal(2)
        
        grad_u = grad(us) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py \
        + coeff_x * grad(px) + coeff_y * grad(py)
        
        grad_v = grad(vs) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy \
        + coeff_x * grad(qx) + coeff_y * grad(qy)
        
        grad_uOther = grad(us.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py.Other() \
        + coeff_x * grad(px.Other()) + coeff_y * grad(py.Other())
        
        grad_vOther = grad(vs.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy.Other() \
        + coeff_x * grad(qx.Other()) + coeff_y * grad(qy.Other())
                
            
        mean_dudn = eps * 0.5 * n * (grad_u + grad_uOther)
        mean_dvdn = eps * 0.5 * n * (grad_v + grad_vOther)
        
        h = specialcf.mesh_size
        a = BilinearForm(fes)
        
        diffusion = eps * grad_u * grad_v * dx \
        + alpha * eps * order**2/h * jump_u * jump_v * dx(skeleton=True) \
        + (-mean_dudn* jump_v - mean_dvdn * jump_u) * dx(skeleton=True) \
        + alpha * eps * order**2/h * u * v * ds(skeleton=True) \
        + eps * (-n * grad_u * v - n * grad_v * u) * ds(skeleton=True)
                
        a += diffusion
        a.Assemble()

 
        f = LinearForm(fes)
        f += SymbolicLFI(exact * v )
        f.Assemble()


        gfu = GridFunction(fes, name="uDG")
        gfu.vec.data = a.mat.Inverse() * f.vec
        
        b = CoefficientFunction((beta[0],beta[1]) )

        uup = IfPos(b*n, u, u_Other)

        convection = -b * u * grad_v * dx + b * n * uup * jump_v * dx(skeleton=True) \
        + IfPos(b*n,b*n,0)*u*v*ds(skeleton=True)

        acd = BilinearForm(fes)
        acd += diffusion + convection

        acd.Assemble()

        gfu = GridFunction(fes)
        gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        edg.loc[len(edg)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('.......................................................................')
        #Draw (gfu)
        

%store edg

Order: 1  Mesh Size: 1.0 L2 Error: 0.00010335206478335451
Order: 1  Mesh Size: 0.5 L2 Error: 0.00012102823054636331
Order: 1  Mesh Size: 0.25 L2 Error: 0.0031654217464059885
Order: 1  Mesh Size: 0.125 L2 Error: 0.0002558146821706225
Order: 1  Mesh Size: 0.0625 L2 Error: 0.0013535333752480287
Order: 1  Mesh Size: 0.0313 L2 Error: 6.90727676666307e-05
.......................................................................
Order: 2  Mesh Size: 1.0 L2 Error: 5.1836741079600704e-05
Order: 2  Mesh Size: 0.5 L2 Error: 0.002099517057863835
Order: 2  Mesh Size: 0.25 L2 Error: 0.0006995174907982338
Order: 2  Mesh Size: 0.125 L2 Error: 0.00020504851413051216
Order: 2  Mesh Size: 0.0625 L2 Error: 0.0004413405294722084
Order: 2  Mesh Size: 0.0313 L2 Error: 0.00181856603447285
.......................................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.00024393543810475698
Order: 3  Mesh Size: 0.5 L2 Error: 0.002804624777302125
Order: 3  Mesh Size: 0.25 L2 Error: 1477

#### Wall function Enrichment with HDG

In [15]:
ehdg = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:        
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dgjumps=True)
        Q = L2(mesh, order=0)
        eps_size = size
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        # Facets. Here the jumps aren't necessary. 
        F = FacetFESpace(mesh, order=order, dirichlet="bottom|left|right|top")
        fes = FESpace([V, F, Qx, Qy], dgjumps = True)

        (us, uhat, px, py), (vs, vhat, qx, qy) = fes.TnT()
        
        #coeff=exact
        p = (coeff_x * px) + (coeff_y * py) 
        q = (coeff_x * qx) + (coeff_y * qy)

        u = us + p
        v = vs + q
        
        jump_u = u-uhat
        jump_v = v-vhat
        
        grad_u = grad(us) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py \
        + coeff_x * grad(px) + coeff_y * grad(py)
        
        grad_v = grad(vs) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy \
        + coeff_x * grad(qx) + coeff_y * grad(qy)
        
        condense = True
        
        h = specialcf.mesh_size
        n = specialcf.normal(mesh.dim)
        dS = dx(element_boundary=True)
                
        diffusion = grad_u * grad_v *dx + \
            alpha * order**2/h*jump_u*jump_v*dS + \
            (-grad_u *n*jump_v - grad_v *n*jump_u)*dS
        
        
        b = CoefficientFunction((beta[0],beta[1]))
        uup = IfPos(b * n, u, uhat)
        convection = -b * u * grad_v *dx + b * n * uup * jump_v * dS

        acd = BilinearForm(fes)
        acd += diffusion + convection
        acd.Assemble()
        
        f = LinearForm(fes)
        f += SymbolicLFI(exact*v)
        f.Assemble()

        gfu = GridFunction(fes)
        ## umfpack gives weird values
        gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky") * f.vec
        u2 = gfu.components[0] + gfu.components[2] * coeff_x + gfu.components[3] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        #edg.loc[len(edg)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('.......................................................................')

%store ehdg    

Order: 1  Mesh Size: 1.0 L2 Error: 5.2474968907730136e-05
Order: 1  Mesh Size: 0.5 L2 Error: 5.231359789415091e-05
Order: 1  Mesh Size: 0.25 L2 Error: 5.1541222786700634e-05
Order: 1  Mesh Size: 0.125 L2 Error: 5.139403139525905e-05
Order: 1  Mesh Size: 0.0625 L2 Error: 5.374170248671766e-05
Order: 1  Mesh Size: 0.0313 L2 Error: 5.0380847276608e-05
.......................................................................
Order: 2  Mesh Size: 1.0 L2 Error: 5.2669701720727525e-05
Order: 2  Mesh Size: 0.5 L2 Error: 5.260467865294426e-05
Order: 2  Mesh Size: 0.25 L2 Error: 4.699658171416195e-05
Order: 2  Mesh Size: 0.125 L2 Error: 5.105754979304879e-05
Order: 2  Mesh Size: 0.0625 L2 Error: 5.045324452474634e-05
Order: 2  Mesh Size: 0.0313 L2 Error: 5.0425863654721814e-05
.......................................................................
Order: 3  Mesh Size: 1.0 L2 Error: 5.200306004624898e-05
Order: 3  Mesh Size: 0.5 L2 Error: 5.238364501918883e-05
Order: 3  Mesh Size: 0.25 L2 Error: 5.